**Connect to Elasticsearch server**

In [3]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')

client_info = es.info()

print("Connected to Elastic Search!")

pprint(client_info.body)

Connected to Elastic Search!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Create Index with auto mappings**

In [4]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [5]:
document = {
    "fast_name": "Shamim",
    "last_name": "Islam"
}

response = es.index(index="my_index", body=document)
response["_id"]

'BZKZPpoBsuuLZ2nEojKT'

Update API
1. If documents exists in the index 
#
1.1 Update an existing field

In [6]:
import json 
from tqdm import tqdm

document_ids = []
documents = json.load(open("data.json"))

for document in tqdm(documents, total=len(documents)):
    response = es.index(index="my_index", body=document)
    document_ids.append(response["_id"])

document_ids

100%|██████████| 5/5 [00:00<00:00,  7.69it/s]


['BpLDPpoBsuuLZ2nEFDJT',
 'B5LDPpoBsuuLZ2nEFjIl',
 'CJLDPpoBsuuLZ2nEFjJU',
 'CZLDPpoBsuuLZ2nEFjKE',
 'CpLDPpoBsuuLZ2nEFjKv']

In [12]:
es.update(
    index="my_index",
    id="BpLDPpoBsuuLZ2nEFDJT",
    script={
        "source": "ctx._source.title = params.title",
        "params": {
            "title": "Updated title by Shamim"
        }
    }
)

ObjectApiResponse({'_index': 'my_index', '_id': 'BpLDPpoBsuuLZ2nEFDJT', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 8, '_primary_term': 1})

In [13]:
response = es.get(index="my_index", id="BpLDPpoBsuuLZ2nEFDJT")
pprint(response.body)

{'_id': 'BpLDPpoBsuuLZ2nEFDJT',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 8,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Updated title by Shamim'},
 '_version': 4,
 'found': True}


1.2 Add a new field
#
To add a new field, you can either use the script argument or the doc argument.
#
1.2.1 Method 1 (Script)

In [14]:
es.update(
    index="my_index",
    id="BpLDPpoBsuuLZ2nEFDJT",
    script={
        "source": "ctx._source.added_new_field_by_shamim = 'dummy_data'"
    }
)

ObjectApiResponse({'_index': 'my_index', '_id': 'BpLDPpoBsuuLZ2nEFDJT', '_version': 5, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 9, '_primary_term': 1})

In [15]:
response = es.get(index="my_index", id="BpLDPpoBsuuLZ2nEFDJT")
pprint(response.body)

{'_id': 'BpLDPpoBsuuLZ2nEFDJT',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 9,
 '_source': {'added_new_field_by_shamim': 'dummy_data',
             'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Updated title by Shamim'},
 '_version': 5,
 'found': True}


1.2.2 Method 2 (doc)

In [16]:
es.update(
    index="my_index", 
    id="BpLDPpoBsuuLZ2nEFDJT",
    doc={
        "new_field_using_doc": "dummy_field"
    }
)

ObjectApiResponse({'_index': 'my_index', '_id': 'BpLDPpoBsuuLZ2nEFDJT', '_version': 6, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 10, '_primary_term': 1})

In [17]:
response = es.get(index="my_index", id="BpLDPpoBsuuLZ2nEFDJT")
pprint(response.body)

{'_id': 'BpLDPpoBsuuLZ2nEFDJT',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 10,
 '_source': {'added_new_field_by_shamim': 'dummy_data',
             'created_on': '2024-09-22',
             'new_field_using_doc': 'dummy_field',
             'text': 'This is the first sample document text.',
             'title': 'Updated title by Shamim'},
 '_version': 6,
 'found': True}


1.3 Remove a field

In [19]:
es.update(
    index="my_index",
    id="BpLDPpoBsuuLZ2nEFDJT",
    script={
        "source": "ctx._source.remove('added_new_field_by_shamim')"
    }
)

ObjectApiResponse({'_index': 'my_index', '_id': 'BpLDPpoBsuuLZ2nEFDJT', '_version': 7, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 11, '_primary_term': 1})

In [20]:
response = es.get(
    index="my_index", 
    id="BpLDPpoBsuuLZ2nEFDJT"
)

pprint(response.body)

{'_id': 'BpLDPpoBsuuLZ2nEFDJT',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 11,
 '_source': {'created_on': '2024-09-22',
             'new_field_using_doc': 'dummy_field',
             'text': 'This is the first sample document text.',
             'title': 'Updated title by Shamim'},
 '_version': 7,
 'found': True}


**2. If documents doesn't exist in the index**
#
We use doc_as_upsert to tell Elasticsearch that if the document does not exist, it should be inserted as a new document.

In [25]:
es.update(
    index="my_index",
    id="1",
    doc={
        "book_id": 1234,
        "new_field": "dummy_data"
    },
    doc_as_upsert=True
)

ObjectApiResponse({'_index': 'my_index', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 12, '_primary_term': 1})

In [26]:
response = es.get(index="my_index", id="1")
pprint(response.body)

{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 12,
 '_source': {'book_id': 1234, 'new_field': 'dummy_data'},
 '_version': 1,
 'found': True}


In [28]:
response = es.count(index="my_index")
count = response["count"] 
count

7